### Contents
##### 1.0 Import of Libraries
##### 2.0 Importing the Dataset 
##### 3.0 Aggregating Data with transform() and creating new variables colum 
##### 3.1 Deriving Columns with loc()


### 1.0 Import of Libraries

In [2]:
import pandas as pd
import numpy as np
import os 

### 2.0 Import Dataset 

In [8]:
#Importing the Dataset 
df_ords_prods_merged = pd.read_pickle('/Users/shaqurcasco/Downloads/ Instacart Basket June: 26: 2023 CF/3 SCRIPT/CF Achievement 4/df_ords_prods_merge_lp.pkl')

In [10]:
df_ords_prods_merged.head()

,order_id,user_id,eval_set,order_number,order_days_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,Busiest day,Busiest days,Hour_of_day
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Average orders
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Most orders


In [14]:
#creating subset of 1 millions row 
df = df_ords_prods_merged[:1000000]

In [15]:
# creating the columns of the subset
df.shape

(1000000, 19)

In [16]:
df.groupby('department_id')

In [17]:
# grouping the department_id by user_id mean 
df.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


In [25]:
#Performing Multiple Aggregations
df.groupby('department_id').agg({'order_number':['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
4                18.825780   1  99
7                17.472355   1  99
13               17.993423   1  99
14               19.246334   1  99
16               19.463012   1  99
17               11.294069   1  98
19               19.305237   1  99
20               17.599636   1  99

###  3.0 Aggregating Data with transform() and creating new variables column

###### Split the data into groups based on the “user_id” column.
###### Apply the transform() function on the “order_number” column to generate the maximum orders for each user.
###### Create a new column, “max_order,” into which you’ll place the results of your aggregation.

In [26]:
# Creating new column['max order'] for loyalty flag for user_id using order_number 
# firsty groupby, aggregate and create a flag
df_ords_prods_merged['max_order'] = df_ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [33]:
df_ords_prods_merged.head(5)

,order_id,user_id,eval_set,order_number,order_days_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,Busiest day,Busiest days,Hour_of_day,max_order
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders,10
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders,10
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Average orders,10
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy days,Most orders,10


### 3.1 Deriving Columns with loc()

In [36]:
# Creating loyalty flag with max order of the user_id > 40 as loyal customer  
df_ords_prods_merged.loc[df_ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [37]:
# Creating loyalty flag with max order of the user_id <= 40 and > 10 as regular customer
df_ords_prods_merged.loc[(df_ords_prods_merged['max_order'] <= 40) 
                     & (df_ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'


In [38]:
# Creating loyalty flag with max order of the user_id <= 10 as new customer
df_ords_prods_merged.loc[df_ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [40]:
# frequency< of each loyally flag
df_ords_prods_merged['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [44]:
# CHECKING THE FIRST  ROWS OF MULTIPLE COLUMNS - USING DOUBLE SQUARE BRACKET 
df_ords_prods_merged[['user_id', 'loyalty_flag', 'order_number']].head(5)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5


 QUESTION 3 :Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code

Question 5 : The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [46]:
df_ords_prods_merged.head(3)

,order_id,user_id,eval_set,order_number,order_days_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,_merge,price_range_loc,Busiest day,Busiest days,Hour_of_day,max_order,loyalty_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer


In [48]:
# checking the price statistic based on the loyalty_flag of the cuatomers
df_ords_prods_merged.groupby('loyalty_flag').agg({'prices':['mean', 'max', 'min']})

prices              
                       mean      max  min
loyalty_flag                             
Loyal customer    10.386336  99999.0  1.0
New customer      13.294670  99999.0  1.0
Regular customer  12.495717  99999.0  1.0

 There are difference between the price of products purchased by loyal customers differ from those purchased by regular or new customers. The price of purchased by New customer is higher , followed by Regular customer and Loyal customer 

#### Question 6 : The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. 
##### Create a spending flag for each user based on the average price across all their orders using the following criteria:
#### If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
##### If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a“High spender.

In [50]:
#Creating new column['Average_spender'] for loyalty flag for user_id using prices 
df_ords_prods_merged['Average_spender'] = df_ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [54]:
df_ords_prods_merged.head(3)

,order_id,user_id,eval_set,order_number,order_days_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,department_id,prices,_merge,price_range_loc,Busiest day,Busiest days,Hour_of_day,max_order,loyalty_flag,Average_spender
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797
2,473747,1,prior,3,3,12,21.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797


In [56]:
### Creating Spending flag for the user_id based on the average spending < 10 = low spender
df_ords_prods_merged.loc[df_ords_prods_merged['Average_spender'] < 10, 'spending_flag'] = 'Low spender'

In [57]:
### Creating Spending flag for the user_id based on the average spending >= 10 = high spender
df_ords_prods_merged.loc[df_ords_prods_merged['Average_spender'] >= 10, 'spending_flag'] = 'High spender'

In [58]:
df_ords_prods_merged.head(3)

,order_id,user_id,eval_set,order_number,order_days_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,_merge,price_range_loc,Busiest day,Busiest days,Hour_of_day,max_order,loyalty_flag,Average_spender,spending_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender
2,473747,1,prior,3,3,12,21.0,196,1,1,...,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender


In [61]:
#  Check frequencies of Average_spending  
df_ords_prods_merged['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

 Question: In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again),the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows 

##### If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a“Non_frequent customer.”
###### If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
######  If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [63]:
# Creating new column['Median_customers'] for frequency_flag for user_id days_since_prior_order
df_ords_prods_merged['Median_customers'] = df_ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [64]:
# Creating frequency_flag for the user_id based on the Median_customers > 20 non_frequent customer
df_ords_prods_merged.loc[df_ords_prods_merged['Median_customers'] > 20, 'frequency_flag'] = 'Non-frequent customer' 

In [69]:
# Creating frequency_flag for the user_id based on the Median_customers > 10 and < = reqular_frequent customer
df_ords_prods_merged.loc[(df_ords_prods_merged['Median_customers'] > 10) & 
                          (df_ords_prods_merged['Median_customers'] <= 20), 'frequency_flag'] = 'Regular customer'

In [70]:
# # Creating frequency_flag for the user_id based on the Median_customers <= 10 frequent customer
df_ords_prods_merged.loc[df_ords_prods_merged['Median_customers'] <= 10, 'frequency_flag'] = 'frequent customer'

In [78]:
df_ords_prods_merged.head(5)

,order_id,user_id,eval_set,order_number,order_days_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,Busiest day,Busiest days,Hour_of_day,max_order,loyalty_flag,Average_spender,spending_flag,Median_customers,frequency_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [73]:
#Check frequencies of spending flags for accuracy
df_ords_prods_merged['frequency_flag'].value_counts(dropna=False)

frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

WE HAVE A FREQUENCY NULL VALUE OF 5, i examine the reason for NAn

In [80]:
# Checking the null values
print(df_ords_prods_merged[df_ords_prods_merged.frequency_flag.isnull()])

          order_id  user_id eval_set  order_number  order_days_of_the_week  \
13645692    895835   159838    prior             1                       0   
17251990    895835   159838    prior             1                       0   
17622767    895835   159838    prior             1                       0   
24138593    895835   159838    prior             1                       0   
25880002    895835   159838    prior             1                       0   

          order_hour_of_day  days_since_prior_order  product_id  \
13645692                 17                     NaN       10749   
17251990                 17                     NaN       33401   
17622767                 17                     NaN       23695   
24138593                 17                     NaN       21334   
25880002                 17                     NaN       22198   

          add_to_cart_order  reordered  ...    price_range_loc  Busiest day  \
13645692                  3          0  ...      

##### The customers with null is becuase there order is not more than one and the median can not be calculated, so the customer can be grouped to 'Non-frequent customer'

In [87]:
#Grouping the customer with Nan into non_frequent customer
df_ords_prods_merged['frequency_flag'].fillna('Non-frequent customer', inplace = True)

In [88]:
#Check frequencies of frequency_flag for accuracy after the customer with null is grouped as non_frequent customer
df_ords_prods_merged['frequency_flag'].value_counts(dropna=False)

frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636442
Name: frequency_flag, dtype: int64

In [90]:
#Assign the ords_prods_merged_gp to df_ords_prods_merged
ords_prods_merged_gp = df_ords_prods_merged

In [93]:
# saving ords_prods_merged_gp.pkl
ords_prods_merged_gp.to_pickle('/Users/shaqurcasco/Downloads/ Instacart Basket June: 26: 2023 CF/3 SCRIPT/CF Achievement 4/ords_prods_merged_gp.pkl')